In [3]:
import torch
torch.cuda.is_available()

/viscam/u/ksarge/mambaforge/envs/u2net/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [39]:
import os
import glob

DATA_PATTERNS = [
    "/viscam/u/ksarge/bike_symbols/*/*/*png",
    "/viscam/u/ksarge/bike_symbols/*/*/*jpg",
    "/viscam/u/ksarge/bike_symbols/*/*/*/*jpg",
    "/viscam/u/ksarge/bike_symbols/*/*/*/*png",
]
LABEL_PATTERNS = [
    "/viscam/u/ksarge/bike_symbols/*/*/*json",
    "/viscam/u/ksarge/bike_symbols/*/*/*/*json",
]

def get_files(patterns):
    files = [
        file
        for pattern in patterns
        for file in glob.glob(pattern)]
    return files

def get_files

In [40]:
len(data_files)

375

clipping_camera.jpg  README.md		      u2net_portrait_composite.py
data_loader.py	     requirements.txt	      u2net_portrait_demo.py
figures		     saved_models	      u2net_portrait_test.py
gradio		     setup_model_weights.py   u2net_test.py
LICENSE		     test_data		      u2net_train.py
model		     testing.ipynb
__pycache__	     u2net_human_seg_test.py
